# Question and answer LLM using the Vector Database data as context

Prerequites: 

1. This notebook assumes you've completed the previous notebook and have data loaded into your vector store. 

2. Azure openAI endpoint
    Confirm that you've deployed both an embedding model and a LLM. https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/create-resource?pivots=web-portal
3. An Azure Machine Learning workspace
    You will need to have an AML workspace provisioned, once you have the workspace and create a compute instance make sure to provision the instance in the same virtual network as your Weaviate instance or your notebook cannot interact with the Vector store. 


In [7]:
%pip install weaviate-client
%pip install langchain
%pip install openai[datalib]
%pip install tiktoken

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 14.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 771.9/771.9 kB 21.0 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


### Set embedding parameters and LLM paramters

In [4]:
# Your deployed embedding model
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    deployment="YOUR_DEPLOYMENT",
    model="YOUR_EMBEDDING_MODEL",
    openai_api_base="YOUR_URL",
    openai_api_type="azure",
    openai_api_key="YOUR_KEY",
    chunk_size = 16
)

In [ ]:
# Your deployed LLM
from langchain.chains import RetrievalQA
from langchain.chat_models import AzureChatOpenAI

llm = AzureChatOpenAI(
    openai_api_base="YOUR_URL",
    openai_api_version="2023-05-15",
    deployment_name="YOUR_LLM",
    openai_api_key="YOUR_KEY",
    openai_api_type="azure",
)

### Set Vector Store parameters 

In [8]:
from langchain.vectorstores import Weaviate
import weaviate
doc = []


WEAVIATE_URL = "YOUR_URL" #example: http://10.244.3.20:8080"
WEAVIATE_API_KEY = "YOUR_KEY"

client = weaviate.Client(url=WEAVIATE_URL, auth_client_secret=weaviate.AuthApiKey(WEAVIATE_API_KEY))

vectorstore = Weaviate.from_documents(doc, embeddings, client=client, by_text=False)

## Data retriever and Questions

The retriever is default to 4 documents and a similarity search. This can be modified: https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.weaviate.Weaviate.html#langchain.vectorstores.weaviate.Weaviate.as_retriever

In [ ]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(llm, retriever=vectorstore.as_retriever())

In [10]:
question = "What is the relationship between quantum Mechanics and computer science, use only the context"

result = qa_chain({"query": question})
answer = result["result"]
print(answer)

The relationship between quantum mechanics and computer science lies in the field of quantum computing. Quantum mechanics provides the theoretical foundation for understanding the behavior of particles at the quantum level, while computer science focuses on the design and analysis of algorithms and computation. Quantum computing explores the use of quantum phenomena, such as superposition and entanglement, to perform computational tasks more efficiently than classical computers. By leveraging the principles of quantum mechanics, quantum computers have the potential to solve certain problems exponentially faster. However, fully realizing the potential of quantum computing requires further advancements in both quantum hardware and software.
